Make decoy datasets 
* input comprised of two sentences with agreeing sentiments in training but disagreeing in test ("This movie is very good. I like it a lot" vs "This movie is very good. The director must have been brain-dead")
* Have random agreeing words in train but disageee in test
* Key idea: structured noise 


In [1]:
%matplotlib inline
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
sys.path.append('../../acd')
sys.path.append('../../acd/visualization')
sys.path.append('../../acd/acd/util')
sys.path.append('../../acd/acd/scores')
sys.path.append('../../acd/acd/agglomeration')

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
from os.path import join
import torch
import torch
import numpy as np
import seaborn as sns
from copy import deepcopy
from model import LSTMSentiment
import matplotlib.pyplot as plt
from os.path import isdir
import viz_1d as viz
import tiling_1d as tiling
import agg_1d as agg
import cd
import score_funcs
import dsets
from dsets.sst import dset
from dsets.sst.model import LSTMSentiment
# check out how two models differ
import torch.optim as O
import torch.nn as nn
from torchtext import data
from torchtext import datasets
from tqdm import tqdm_notebook, tqdm 
import pickle

In [2]:
sst_pkl = pickle.load(open('../../acd/dsets/sst/sst.pkl', 'rb'))
np.random.seed(42)

In [3]:

vector_cache = os.path.join(os.getcwd(), '../data/.vector_cache/input_vectors.pt')
word_vectors ='glove.6B.300d'
batch_size=  50

In [105]:
# load test set

inputs = data.Field(lower= True)
answers = data.Field(sequential=False, unk_token=None)

train, dev, test = datasets.SST.splits(inputs, answers, fine_grained=False, train_subtrees=True,
                                       filter_pred=lambda ex: ex.label != 'neutral')

inputs.build_vocab(train, dev, test)

In [106]:
len(train)

98794

In [83]:
if word_vectors:
    if os.path.isfile(vector_cache):
        inputs.vocab.vectors = torch.load(vector_cache)
    else:
        inputs.vocab.load_vectors(word_vectors)
        makedirs(os.path.dirname(vector_cache))
        torch.save(inputs.vocab.vectors,vector_cache)
answers.build_vocab(train)
train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_size=batch_size, device=torch.device(0))

In [84]:
len(train)

3310

In [100]:
length = np.empty((4000))
for idx, batch in tqdm_notebook(enumerate(train_iter)):
    
    length[idx] = (batch.text.shape[0])
    break

In [102]:
batch.text.shape

torch.Size([42, 50])

In [107]:
for i in range(100):
    print(" ".join(train.examples[i].text))

the rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal
destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal
new
that he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal
he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal
's going to make a splash even greater than arnold schwarzenegger , jean-claud va

In [76]:
train.examples[2].text

['more', 'simply', 'intrusive', 'to', 'the', 'story']